In [1]:
import re

def team_name(input_string):
    """
    Strips the ranking component (a number in parentheses at the end) from the given string.
    """
    # Use regex to match and strip "(number)" only if it appears at the end
    return re.sub(r'\s*\(\d+\)$', '', input_string)

# Examples
assert team_name("UConn (6)") == "UConn"
assert team_name("North Carolina") == "North Carolina"
assert team_name("UConn (North) (9)") == "UConn (North)"
assert team_name("UConn (North)") == "UConn (North)"

In [2]:
import pandas as pd
file_name = "Basketball_dataset.xlsx"
excel_file = pd.ExcelFile(file_name)

team_metadata = excel_file.parse(sheet_name="Teams")
team_metadata["Team"] = team_metadata["Team"].apply(lambda s : s.strip())

sheet_names = {
    team_name(sheet_name) : sheet_name for sheet_name in excel_file.sheet_names[1:]
}

In [3]:
"""
Each row in our dataframe of all the games looks as follows:


Team,Type,Opponent,For,Against
"""
def resolve_team_name(sheet_name):
    """
    Resolve the true name of a team by examining ranked opponents' sheets
    Why do we need this? UConn is represented by Connecticut in the opponent's table

    Args:
        sheet_name (str): The current sheet name.
        ranked_teams (set): Set of ranked team names.
        excel_file (pd.ExcelFile): The Excel file object.

    Returns:
        str: The resolved "true name" of the team.
    """
    # Try to parse the current team's sheet
    our_team_sheet = excel_file.parse(sheet_name=sheet_name)
    opponents = our_team_sheet["Opponent"].dropna().apply(team_name)

    # Check if any opponent is a ranked team
    for i,row in our_team_sheet.dropna().iterrows():
        opponent_name = team_name(row["Opponent"])
        if opponent_name in sheet_names and sheet_names[opponent_name] in excel_file.sheet_names:
          opponent_sheet = excel_file.parse(sheet_name=sheet_names[opponent_name])
          target_row = opponent_sheet[opponent_sheet.apply(lambda r : r["Date"] == row["Date"], axis=1)]
          if not target_row.empty:
            # Extract the "Team" column from the opponent's perspective
            resolved_name = target_row.iloc[0]["Opponent"]
            return team_name(resolved_name)     

    print(f"Team: {sheet_name}, Unable to identify team's 'real' name, they didn't play against any ranked opponents that we could identify")
    return team_name(sheet_name)

abbrev_to_team_name = {
    team : resolve_team_name(sheet_names[team]) for team in sheet_names
}
team_names = set(abbrev_to_team_name.values())
print('San Diego State' in team_names)


Team: Duke(13), Unable to identify team's 'real' name, they didn't play against any ranked opponents that we could identify
True


In [4]:
def combine_basketball_sheets():
    """
    Combine multiple sheets from an Excel file into a single DataFrame with filtering.

    Returns:
        pd.DataFrame: Combined DataFrame with columns Type, Team, Opponent, For, Against.
    """

    combined_data = []

    for sheet_name in excel_file.sheet_names[1:]:
        team_data = excel_file.parse(sheet_name=sheet_name)

        our_name = abbrev_to_team_name[team_name(sheet_name)]
        team_data["Team"] = our_name
        team_data.rename(columns={"Tm": "For", "Opp": "Against"}, inplace=True)
        team_data["Opponent"] = team_data["Opponent"].dropna().apply(team_name)
    
        team_data = team_data[["Date", "Type", "Team", "Opponent", "For", "Against"]]
        combined_data.append(team_data)

    # Concatenate all data into a single DataFrame
    df = pd.concat(combined_data, ignore_index=True).dropna()

    ### Drop duplicates (just add columns that uniquely identify a game )

    # Ensure consistent ordering for Team/Opponent and For/Against
    df["Team_Ordered"] = df[["Team", "Opponent"]].min(axis=1)
    df["Opponent_Ordered"] = df[["Team", "Opponent"]].max(axis=1)
    df["For_Ordered"] = df[["For", "Against"]].min(axis=1)
    df["Against_Ordered"] = df[["For", "Against"]].max(axis=1)

    # Drop duplicates based on ordered columns
    df = df.drop_duplicates(subset=["Team_Ordered", "Opponent_Ordered", "For_Ordered", "Against_Ordered", "Type", "Date"])

    # Drop the helper columns
    df = df.drop(columns=["Team_Ordered", "Opponent_Ordered", "For_Ordered", "Against_Ordered"])


    return df

all_games = combine_basketball_sheets()

# INCLUDES NON_RANKED_TEAMS! PLEASE USE THIS ONE
all_teams = set(all_games["Team"]).union(set(all_games["Opponent"]))
# UNIQUE ID FOR EACH TEAM
id_to_team = { i:team for i,team in enumerate(all_teams)}
num_teams = len(all_teams)

In [5]:
from functools import cache

@cache
def games_of(team_name):
  games_by = all_games[all_games.apply(lambda r: r["Team"] == team_name or r["Opponent"] == team_name, axis=1)]
  #print(games_by)
  def normalize_game(row):
    if row["Team"] == team_name:
      return row  # No changes needed
    else:
            # Swap Team and Opponent, For and Against
      return {
                "Type": row["Type"],
                "Team": row["Opponent"],
                "Opponent": row["Team"],
                "For": row["Against"],
                "Against": row["For"],
            }

    # Apply normalization to ensure Team == team_name
  return games_by.apply(normalize_game, axis=1, result_type="expand")

games_of_cached = {
    id : games_of(team) for id,team in id_to_team.items()
}

In [6]:

def more_wins_model(all_games):
  return {
      team : sum(1 for _,game in games_of(team).iterrows() if game["For"] > game["Against"]) for team in all_teams
  }

import random

def random_model(all_games):
  return {
      team : random.uniform(0, 1) for team in all_teams
  }

In [7]:
from scipy.optimize import minimize
import numpy as np

def bradley_terry_model(all_games):
    small_team_count = num_teams
    # Precompute win_difference
    win_difference = {}
    for i, team1 in id_to_team.items():
        for j, team2 in id_to_team.items():
            team1_games = games_of(team1)
            opposing_games = team1_games[team1_games["Opponent"] == team2]
            wins = sum(opposing_games["For"] > opposing_games["Against"])
            losses = len(opposing_games) - wins
            win_difference[(i, j)] = wins - losses

    win_diff_values = np.array([win_difference[(i, j)] for i in range(small_team_count) for j in range(small_team_count)])

    # Optimized objective function
    def optimized_objective(alpha):
        return sum(
            np.log(alpha[i] / (alpha[i] + alpha[j])) ** win_difference[(i,j)]
            for i in range(num_teams) for j in range(num_teams))

    # Set up optimization
    num_params = small_team_count
    initial_guess = np.ones(num_params) 
    result = minimize(
            optimized_objective, 
            initial_guess, 
            method="L-BFGS-B", 
            bounds=[(1e-6, 1)] * num_params, 
            tol=1e-5,          
            options={"maxiter": 1000}) 

    return result

In [8]:
"""
Train on the preseason games, test on the march madness games.

Of all march madness games, how many do we correctly predict?
"""
def evaluate_model(model, model_name):
  is_tournament_game = lambda r : r["Type"] != "NCAA"
  training_set = all_games[all_games.apply(is_tournament_game, axis=1)]
  testing_set =  all_games[~all_games.apply(is_tournament_game, axis=1)]

  weights = model(training_set)

  correct_guesses = 0
  total_guesses = 0

  for _, r in testing_set.iterrows():
    exp_result = weights[r["Team"]] > weights[r["Opponent"]]
    seen_result = r["For"] > r["Against"]
    correct_guesses += exp_result == seen_result
    total_guesses += 1


  print(f'{model_name}, Predicted {correct_guesses} of {total_guesses} games correctly')

In [ ]:
evaluate_model(more_wins_model, "more wins")
evaluate_model(random_model, "random")
evaluate_model(bradley_terry_model, "bradley terry")